# Cube Packcage
The custom cube package is preinstalled.  just load the library and start to use it 
<hr/>


### Load Library and Login to CubeAPI

In [1]:
# load all library, it is already installed
library(devtools)
library(httr)
library(jsonlite)
library(stringr)
library(rapportools)
library(htmlwidgets)
library(rlist)
library(data.table)
library(cloudml)
library(readr)
library(logger)
library(cube)
setwd("~/")
log_threshold(INFO)

Loading required package: usethis

Loading required package: reshape


Attaching package: ‘rapportools’


The following objects are masked from ‘package:stats’:

    IQR, median, sd, var


The following objects are masked from ‘package:base’:

    max, mean, min, range, sum



Attaching package: ‘data.table’


The following object is masked from ‘package:reshape’:

    melt


Loading required package: tfruns



In [2]:
# create a CubeAPI object
cube_api = CubeAPI$new()

INFO [2021-01-08 21:35:51] Version: 0.0.2
INFO [2021-01-08 21:35:51] http://10.105.16.22/metadata-service/


In [3]:
# only need to login once per month
# after run, click on "verification_uri_complete" to finish verification
cube_api$login()

INFO [2021-01-08 21:35:52] verification_uri_complete: https://thejacksonlaboratory.auth0.com/activate?user_code=CMQF-MSGV


### 1. Get Metadata Definition

In [3]:
# retrieve metadata element
response = cube_api$get_element()

INFO [2021-01-14 13:29:03] GET: http://10.105.16.22/metadata-service/metadata_definition/element/
INFO [2021-01-14 13:29:04] status_code: 200


In [ ]:
str(response)

In [ ]:
# get the content of the response
content = content(response)

In [ ]:
str(content)

In [4]:
# to data frame
data.frame(do.call(rbind, content$results))[c("id", "element_label", "element_collection_label", "element_abbreviation")]

ERROR: Error in content$results: object of type 'closure' is not subsettable


### 2. Get Bucket data from metadata collection

In [4]:
storage_info = cube_api$get_metadata_collection_storage_info()


INFO [2021-01-08 21:36:00] GET: http://10.105.16.22/metadata-service/metadata_repository/collection/
INFO [2021-01-08 21:36:01] status_code: 200
INFO [2021-01-08 21:36:01] number of accession_ids: 1
INFO [2021-01-08 21:36:01] GET: http://10.105.16.22/metadata-service/metadata_repository/element_instance/
INFO [2021-01-08 21:36:02] status_code: 200
INFO [2021-01-08 21:36:02] number of row returned: 2


In [5]:
storage_info

,accession_id,uri,bucket_name,file_name
,<chr>,<chr>,<chr>,<chr>
1,JAXDS0000G,gs://jax-cube-prd-ctrl-01-project-data/study_3-1/Three_bears_CBA_BODY_WEIGHT_200723.txt,gs://jax-cube-prd-ctrl-01-project-data/study_3-1,Three_bears_CBA_BODY_WEIGHT_200723.txt
2,JAXDS0000G,gs://jax-cube-prd-ctrl-01-project-data/20191223_19-churchill-004,gs://jax-cube-prd-ctrl-01-project-data/20191223_19-churchill-004,


In [ ]:
!CubeAPI

In [ ]:
# pick a bucket
bucket_name = storage_info[1, 3]
bucket_name

In [ ]:
# list out files in a bucket
system(str_glue('gsutil ls -lh {bucket_name}'), intern = TRUE)

In [ ]:
# pick a file to copy to workspace
file_name = "Run_Metric_Summary_20-cube-001-run5.csv"
setwd("~/")
system(str_glue('gsutil cp {bucket_name}/{file_name} {file_name} 2>&1'), intern = TRUE)

In [ ]:
# read data into a data frame
read.csv(file_name)

In [ ]:
## sync a entire bucket directory, (bucket could be huge) 
# bucket_data = get_bucket_data(bucket_name = bucket_name, file_name = file_name )
# bucket_data

### 3. Get Metadata Definition with a Parameter

In [ ]:
response = cube_api$get_element(element_id = 85)

In [ ]:
str(content(response))

In [ ]:
?system

### 4. Get Metadata with accession ids 

In [ ]:
response = cube_api$get_element_instance(element_id = 85, accession_ids = c("JAXAS00031", "JAXAS00032", "JAXAS00034"))

In [ ]:
str(content(response)$results)

### 5. Get Metadata Collection

In [6]:
response = cube_api$get_metadata_collection()

INFO [2021-01-08 21:36:15] GET: http://10.105.16.22/metadata-service/metadata_repository/collection/
INFO [2021-01-08 21:36:16] status_code: 200


In [7]:
json = response_json_to_data(response)

No encoding supplied: defaulting to UTF-8.



In [8]:
str(json$results)

'data.frame':	1 obs. of  5 variables:
 $ id              : int 10
 $ collection_items:List of 1
  ..$ :'data.frame':	1 obs. of  7 variables:
  .. ..$ id           : int 346
  .. ..$ collection   :'data.frame':	1 obs. of  4 variables:
  .. .. ..$ id             : int 10
  .. .. ..$ collection_name: logi NA
  .. .. ..$ user_name      : chr "waad.939JXKnVNUKLJTw6JEVLxgIlZoYOeIfrJ58NLiCbVQc"
  .. .. ..$ date_created   : chr "2020-12-22T14:24:16.893718Z"
  .. ..$ collection_id: int 10
  .. ..$ accession_id : chr "JAXDS0000G"
  .. ..$ item_type    : chr "Data Set"
  .. ..$ item_label   : logi NA
  .. ..$ date_created : chr "2021-01-08T21:09:00.735434Z"
 $ user_name       : chr "waad.939JXKnVNUKLJTw6JEVLxgIlZoYOeIfrJ58NLiCbVQc"
 $ collection_name : logi NA
 $ date_created    : chr "2020-12-22T14:24:16.893718Z"


In [9]:
accession_ids = json$results[,2][[1]]$accession_id
accession_ids

[1] "JAXDS0000G"

In [10]:
response = cube_api$get_element_instance(accession_ids = c(accession_ids))

INFO [2021-01-08 21:36:33] GET: http://10.105.16.22/metadata-service/metadata_repository/element_instance/
INFO [2021-01-08 21:36:34] status_code: 200


In [12]:
results = content(response)$results
results[[21]][[1]]

ERROR: Error in results[[21]]: subscript out of bounds


In [11]:
data_url = results[[21]][[1]][[1]][[4]]
url_parts = str_split(data_url, "/")
bucket_name = paste0("gs://", url_parts[[1]][4], "/", url_parts[[1]][5])
file_name =  url_parts[[1]][6]

# load the data from the GCP bucket
# data_dir = gs_data_dir( bucket_name )

# read the file into a dataframe
# df <- read.table(file.path(data_dir, file_name), sep = '\t',header = TRUE)
# df

ERROR: Error in eval(expr, envir, enclos): object 'results' not found


### 5. Call Any Get endpoint
Call any get end point.  All the endpoints are listed here: https://metadata-service-sqa.azurewebsites.net/swagger/

In [ ]:
# specify a end point
response = cube_api$get_end_point("metadata_definition/property/?element_id=85")

In [ ]:
content(response)$count

In [ ]:
# another end point
response = cube_api$get_end_point("/metadata_repository/collection_item/1/")

In [ ]:
content(response)

### 6. Post endpoint

In [ ]:
response = cube_api$post_element_instance_filter(element_id = 122, page = 1, page_size = 2)

In [ ]:
str(content(response))

### 7. More Information

In [ ]:
?CubeAPI

In [ ]:
?Cube